# **Importamos las dependencias**

Estoy importando todas las dependencias de mis cuadernos, despues las depuro

In [1]:
# --- Importaciones para Manejo de Archivos y Directorios ---
import os
import sys
import subprocess
import re

# --- Importamos Vertex AI ---
import vertexai


# --- Importaciones para Visualización y Formato de Texto ---

# Se utiliza para mostrar contenido enriquecido, como texto con formato Markdown,
# directamente en entornos como Jupyter Notebooks o IPython.
from IPython.display import Markdown, display

# Importa la clase `Markdown` de la biblioteca `rich`, que sirve para renderizar
# Markdown con formato avanzado en la terminal. Se le da un alias `RichMarkdown`
# para evitar conflictos de nombre con la importación anterior.
from rich.markdown import Markdown as RichMarkdown


# --- Importaciones para el Modelo Generativo de Vertex AI ---

# Importa las clases necesarias del SDK de Vertex AI para interactuar con los modelos generativos.
# - GenerationConfig: Para configurar los parámetros de la respuesta (ej. temperatura, top_p).
# - GenerativeModel: La clase principal para cargar y usar un modelo generativo como Gemini.
# - Image: Para manejar y enviar imágenes como parte de la entrada al modelo (enfoque multimodal).
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image

In [2]:
# --- Para el proceso de datos y visualización ---
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import time


# --- Para manejar las variables de entorno ---
import os
from dotenv import load_dotenv


# --- Desactiva las advertencias de asignaciones encadenadas en pandas para evitar mensajes de warning al modificar DataFrames.
pd.options.mode.chained_assignment = None  # default='warn'


# --- Dependencias de Vertex AI ---
import vertexai                                              # Importa el módulo principal de Vertex AI.
from vertexai import init                                    # Inicializa Vertex AI con las credenciales y configuraciones necesarias.
from vertexai.vision_models import Image as VMImage          # Importa la clase Image de Vertex AI para manejar imágenes.
from vertexai.vision_models import MultiModalEmbeddingModel  # Importa el modelo de embeddings multimodales de Vertex AI para procesar imágenes y videos.
from vertexai.vision_models import Video                     # Clase para manejar archivos de video en Vertex AI.
from vertexai.vision_models import VideoSegmentConfig        # Configuración para segmentar videos
from vertexai.generative_models import GenerativeModel       # Importa la clase para modelos generativos, como Gemini.
from vertexai.generative_models import Part                  # Importa la clase Part para manejar partes de un mensaje, como texto o imágenes.


# --- Para conectarse y consultar un endpoint de búsqueda vectorial (Vector Search) en Vertex AI. 
from google.cloud.aiplatform.matching_engine import MatchingEngineIndexEndpoint 

# --- Para acceder a los buckets de Google Cloud Storage y manejar archivos.
from google.cloud import storage


# --- Dependencias para poder visualizar ---
from IPython.display import Video as MVideo                  # Permite mostrar videos directamente en celdas de Jupyter Notebook.
from IPython.display import HTML                             # Permite mostrar contenido HTML en celdas de Jupyter Notebook.
from IPython.display import Image as ImageByte               # Permite mostrar imágenes en el notebook (renombrado como ImageByte para evitar conflictos de nombres).
from IPython.display import display                          # Función general para mostrar objetos en el notebook.
from sklearn.metrics.pairwise import cosine_similarity       # Función para calcular la similitud coseno entre vectores, útil para comparar embeddings.

# **Configuración de credenciales**

In [3]:
# --- Carga las Variables de Entorno ---
load_dotenv()

PROJECT_ID = os.getenv("PROJECT_ID")                        # ID del proyecto de Google Cloud
LOCATION = os.getenv("LOCATION")                            # Región donde se encuentran los recursos de Vertex AI
INDEX_ID = os.getenv("INDEX_ID")                            # ID del índice de búsqueda vectorial en Vertex AI
ENDPOINT_ID = os.getenv("ENDPOINT_ID")                      # ID del endpoint de búsqueda vectorial en Vertex AI
BUCKET_NAME = os.getenv("BUCKET_NAME")                      # Nombre del bucket de Google Cloud Storage donde se almacenan los videos
VIDEO_FOLDER_PATH = os.getenv("VIDEO_FOLDER_PATH")          # Ruta del folder dentro del bucket donde se encuentran los videos

# Verificamos que las variebles de entorno esten bien
print(f"Proyecto: {PROJECT_ID}, Ubicación: {LOCATION}, Índice: {INDEX_ID}, Endpoint: {ENDPOINT_ID}, Bucket: {BUCKET_NAME}, Ruta de Videos: {VIDEO_FOLDER_PATH}")

Proyecto: constant-setup-463820-p6, Ubicación: us-central1, Índice: 1206368764833038336, Endpoint: mexicocostarica_1750850443317, Bucket: vboxiooof, Ruta de Videos: Videos/Videos_Segmentados


In [29]:
# Iniciamos Vertex AI con el proyecto y la ubicación especificados.
init(project = PROJECT_ID, location = LOCATION)

# **Configuración de Vertex AI**

In [ ]:
# --- Inicialización del LLM ---
model = GenerativeModel('gemini-2.5-flash-lite-preview-06-17')
embedding_model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")

# **Funciones**

In [ ]:
def buscar_videos_similares(texto_de_la_pregunta, cantidad_de_resultados=5):

    """
    Hacemos el embedding de la pregunta, y buscamos en Vector Search los videos más similares.
    Devolvemos las URIs de los videos encontrados.

    Args:
        texto_de_la_pregunta (str): La pregunta o consulta para buscar videos similares.
        cantidad_de_resultados (int): Número de resultados a devolver. Por defecto es 5.

    Returns:
        list: Una lista de URIs de los videos encontrados que son similares a la pregunta.
    """


    print(f'--- 1.- Hacemos el embedding de la pregunta: {texto_de_la_pregunta} ---')

    # Hacemos el embedding de la pregunta
    try:
        vector_de_la_pregunta = embedding_model.get_embeddings(
            contextual_text = texto_de_la_pregunta
        ).text_embedding

    except Exception as e:
        print(f'Error al hacer el embedding de la pregunta: {e}')
        return []
    
    print('Fin del paso 1.\n')


    print(f'\n --- 2.- Nos conectamos a Vector Search ---')

    my_index_endpoint = MatchingEngineIndexEndpoint(
        INDEX_ID = ENDPOINT_ID
    )

    print('Fin del paso 2.\n')



    print(f'\n --- 3.- Buscamos los {cantidad_de_resultados} videos más similares a la pregunta ---')

    try:
        # Buscamos usando la técnica de 'nearest neighbors'
        response = my_index_endpoint.find_neighbors(

            queries = [vector_de_la_pregunta],      # Usamos el embedding de la pregunta
            ENDPOINT_ID = INDEX_ID,           # ID del índice desplegado
            num_neighbors = cantidad_de_resultados  # Número de vecinos a buscar

        )

        # Extraemos las URIs de los videos encontrados
        uris_completas = []

        if response and response[0]:
            for neighbor in response[0]:
                segment_id_base = neighbor.id # Nos da por ejemplo: "mexicosta_segment_1"

                # Extraemos el prefijo y el número del ID
                match = re.match(r"(.*_)(\d+)", segment_id_base)
                if match:
                    prefijo = match.group(1) # "mexicosta_segment_"
                    numero = match.group(2)  # "1" 
                    
                    # Formateamos el número a 3 dígitos con ceros a la izquierda (ej. "001")
                    numero_formateado = numero.zfill(3)
                    
                    # Reconstruimos el nombre del archivo final
                    nombre_archivo_final = f"{prefijo}{numero_formateado}"
                    
                    # Creamos la URI completa con la extensión .mkv
                    uri = f"gs://{BUCKET_NAME}/{VIDEO_FOLDER_PATH}/{nombre_archivo_final}.mkv"
                    uris_completas.append(uri)
                
    
    except Exception as e:
        print(f'Error al buscar los videos similares: {e}')
        return []

In [7]:
def analizar_fragmentos_con_gemini(
    pregunta_original: str,
    uris_de_videos: list[str]
) -> str:
    """
    Toma la pregunta y una lista de URIs de video, las envía a Gemini
    y devuelve la respuesta generada por el modelo.
    """
    if not uris_de_videos:
        return "No se encontraron videos relevantes para analizar."

    print(f"\nPASO 3.1: Preparando {len(uris_de_videos)} videos para enviar a Gemini...")
    
    # Convertimos cada URI de video en un objeto 'Part' que Gemini entiende
    video_parts = [Part.from_uri(uri, mime_type="video/mkv") for uri in uris_de_videos]

    # Construimos el prompt para el modelo
    prompt_completo = [
        "Eres un asistente experto en análisis de video.",
        "Tu tarea es analizar los siguientes fragmentos de video que te proporciono y responder a la pregunta del usuario de la forma más detallada posible basándote ÚNICAMENTE en el contenido de estos videos.",
        "\n---",
        "PREGUNTA DEL USUARIO:",
        pregunta_original,
        "\n---",
        "FRAGMENTOS DE VIDEO A ANALIZAR:",
        *video_parts
    ]

    print("PASO 3.2: Enviando la solicitud a Gemini... (Esto puede tardar un poco)")
    try:
        # Enviamos el prompt completo al modelo generativo
        response = model.generate_content(prompt_completo)
        return response.text
    except Exception as e:
        return f"Ocurrió un error al contactar con el modelo Gemini: {e}"

# **Pruebas**

In [ ]:
# --------------------------------------------------------------------------
# --- SCRIPT FINAL (VERSIÓN 3 - CORREGIDO CON CÓDIGO DE LA CONSOLA) ---
# --------------------------------------------------------------------------
import os
import re
from dotenv import load_dotenv
import vertexai
from google.cloud import aiplatform
from vertexai.generative_models import GenerativeModel, Part
from vertexai.vision_models import MultiModalEmbeddingModel
from IPython.display import Markdown, display

load_dotenv()

# --------------------------------------------------------------------------
# --- 1. CONFIGURACIÓN Y CLIENTES (¡CON LOS IDs EXACTOS!) ---
# --------------------------------------------------------------------------

# --- Constantes para la Configuración de Vertex AI Search (DE TU CÓDIGO) ---
# ¡CORRECCIÓN FINAL! Usamos el nombre de recurso completo del endpoint.
INDEX_ENDPOINT_RESOURCE_NAME = "projects/640283206292/locations/us-central1/indexEndpoints/1468492336894836736"
# ¡CORRECCIÓN FINAL! Usamos el ID del índice desplegado que te dio la consola.
ENDPOINT_ID = "mexicocostarica_1750850443317"

# --- Constantes de tu Bucket (se mantienen igual) ---
BUCKET_NAME = "vboxiooof"
VIDEO_FOLDER_PATH = "Videos/Videos_Segmentados"

# --- Inicialización de Clientes de Vertex AI ---
PROJECT_ID = os.getenv("PROJECT_ID")
LOCATION = os.getenv("LOCATION")

if not PROJECT_ID or not LOCATION:
    raise ValueError("Asegúrate de que GCP_PROJECT_ID y GCP_REGION están en tu archivo .env")

aiplatform.init(project=PROJECT_ID, location=LOCATION)

embedding_model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")
generative_model = GenerativeModel("gemini-2.0-flash-lite-001")


# --------------------------------------------------------------------------
# --- 2. FUNCIÓN DE BÚSQUEDA (El código interno no cambia) ---
# --------------------------------------------------------------------------
def buscar_videos_similares(
    texto_de_la_pregunta: str,
    cantidad_de_resultados: int = 5
) -> list[str]:
    """
    Toma una pregunta, la convierte en un embedding, busca en Vertex AI Search
    los IDs de videos más similares, reconstruye sus URIs de GCS y las devuelve.
    """
    print(f"PASO 2.1: Creando embedding para la pregunta: '{texto_de_la_pregunta}'")
    try:
        vector_de_la_pregunta = embedding_model.get_embeddings(
            contextual_text=texto_de_la_pregunta
        ).text_embedding
    except Exception as e:
        print(f"Error al generar embedding de texto: {e}")
        return []

    print("PASO 2.2: Conectando al endpoint de Vertex AI Search...")
    # Esta línea ahora usará el nombre de recurso completo.
    my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(
        INDEX_ID=INDEX_ENDPOINT_RESOURCE_NAME
    )

    print(f"PASO 2.3: Buscando {cantidad_de_resultados} videos similares...")
    try:
        # Esta línea ahora usará el ENDPOINT_ID correcto.
        response = my_index_endpoint.find_neighbors(
            queries=[vector_de_la_pregunta],
            ENDPOINT_ID=ENDPOINT_ID,
            num_neighbors=cantidad_de_resultados,
        )

        uris_completas = []
        if response and response[0]:
            for neighbor in response[0]:
                segment_id_base = neighbor.id
                match = re.match(r"(.*_)(\d+)", segment_id_base)
                if match:
                    prefijo, numero = match.groups()
                    numero_formateado = numero.zfill(3)
                    nombre_archivo_final = f"{prefijo}{numero_formateado}"
                    uri = f"gs://{BUCKET_NAME}/{VIDEO_FOLDER_PATH}/{nombre_archivo_final}.mkv"
                    uris_completas.append(uri)

        print(f"PASO 2.4: ¡Éxito! URIs encontradas: {uris_completas}")
        return uris_completas

    except Exception as e:
        print(f"Ocurrió un error al consultar Vertex AI Search: {e}")
        return []

# --------------------------------------------------------------------------
# --- 3. FUNCIÓN DE ANÁLISIS CON GEMINI (Sin cambios) ---
# --------------------------------------------------------------------------
def analizar_fragmentos_con_gemini(
    pregunta_original: str,
    uris_de_videos: list[str]
) -> str:
    if not uris_de_videos:
        return "No se encontraron videos relevantes para analizar."

    print(f"\nPASO 3.1: Preparando {len(uris_de_videos)} videos para enviar a Gemini...")
    video_parts = [Part.from_uri(uri, mime_type="video/mkv") for uri in uris_de_videos]
    prompt_completo = [
        "Eres un asistente experto en análisis de video.",
        "Tu tarea es analizar los siguientes fragmentos de video que te proporciono y responder a la pregunta del usuario de la forma más detallada posible basándote ÚNICAMENTE en el contenido de estos videos.",
        "\n---",
        "PREGUNTA DEL USUARIO:",
        pregunta_original,
        "\n---",
        "FRAGMENTOS DE VIDEO A ANALIZAR:",
        *video_parts
    ]
    print("PASO 3.2: Enviando la solicitud a Gemini... (Esto puede tardar un poco)")
    try:
        response = generative_model.generate_content(prompt_completo)
        return response.text
    except Exception as e:
        return f"Ocurrió un error al contactar con el modelo Gemini: {e}"


In [13]:

# --------------------------------------------------------------------------
# --- 4. EJECUCIÓN DEL FLUJO COMPLETO (Sin cambios) ---
# --------------------------------------------------------------------------
if __name__ == "__main__":
    pregunta_del_usuario = "Muestrame a la marca Caliente"
    print("--- INICIANDO PROCESO DE RAG MULTIMODAL ---")
    uris_relevantes = buscar_videos_similares(pregunta_del_usuario)
    respuesta_final = analizar_fragmentos_con_gemini(pregunta_del_usuario, uris_relevantes)
    print("\n--- RESPUESTA FINAL DE GEMINI ---")
    print(respuesta_final)

--- INICIANDO PROCESO DE RAG MULTIMODAL ---
PASO 2.1: Creando embedding para la pregunta: 'Muestrame a la marca Caliente'
PASO 2.2: Conectando al endpoint de Vertex AI Search...
PASO 2.3: Buscando 5 videos similares...
PASO 2.4: ¡Éxito! URIs encontradas: ['gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_1095.mkv', 'gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_167.mkv', 'gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_910.mkv', 'gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_909.mkv', 'gs://vboxiooof/Videos/Videos_Segmentados/mexicosta_segment_1055.mkv']

PASO 3.1: Preparando 5 videos para enviar a Gemini...
PASO 3.2: Enviando la solicitud a Gemini... (Esto puede tardar un poco)

--- RESPUESTA FINAL DE GEMINI ---
Ocurrió un error al contactar con el modelo Gemini: 400 Request contains an invalid argument.
